# Задание 2. Итерационные методы решения линейных систем


In [1]:
from latexifier import latexify
from IPython.display import display, Markdown, Latex
import numpy as np
import pandas as pd
from typing import Callable, List, Tuple, Union, Any
import warnings
warnings.filterwarnings('ignore')


In [2]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()

    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

    def norm(self) -> float:
        """Calculates Matrix norm using `np.linalg.norm` with parameter `np.inf`"""
        return np.linalg.norm(self, np.inf)

    def inv(self) -> Any:
        return Matrix(np.linalg.inv(self))


def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))


In [3]:
def Gauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    """
    Вычисляет решения линейного уравнения Ax = B методом Гаусса с выбором
    главного элемента по столбцам (с перестановкой строк)
    """
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n):
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))


In [4]:
def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg,
                   np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_latex_column(arg: str, n: int) -> str:
    """
    Prepares a string with LaTex syntax for a column of `arg` with indeces form 1 to `n`
    """
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def print_gauss_output(A: Matrix, B: Matrix, X: Matrix, n: int, name: str) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = A @ X
    print_tex(r'\text{', name, '}~X^* = ',
              get_latex_column("x^*", n), '=', get_exact_column(X, n))
    print_tex(rf'A \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n),
              r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


## Вариант 12


In [52]:
n = 4
A = Matrix(np.random.rand(n, n))
B = Matrix(np.random.rand(n, 1))
for i in range(n):
    for j in range(n):
        A[i][j] = A[j][i]
    A[i][i] += np.sum([A[i][j] for j in range(n) if i != j])
print_tex('A=', A, ',~~', 'B=', B)


$$ A= \begin{pmatrix} 2.803282 & 0.884316 & 0.576430 & 0.998670 \\ 0.884316 & 2.447266 & 0.600491 & 0.544755 \\ 0.576430 & 0.600491 & 2.995944 & 0.868780 \\ 0.998670 & 0.544755 & 0.868780 & 2.802353 \end{pmatrix} ,~~ B= \begin{pmatrix} 0.752782 \\ 0.202033 \\ 0.403762 \\ 0.298690 \end{pmatrix} $$

In [53]:
eigenvals = Matrix(np.linalg.eigvals(A))
print('Собственные числа матрицы A:', eigenvals)

Собственные числа матрицы A: [5.03041718 1.57479715 2.40328449 2.04034631]


Найдем решение $x^*$ методом Гаусса.


In [54]:
X_Gauss = Gauss(A, B, n)
print_gauss_output(A, B, X_Gauss, n, 'Решение методом Гаусса:')


$$ \text{ Решение методом Гаусса: }~X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix}
0.263299596030353\\
-0.033386440745686745\\
0.09363947991217723\\
-0.009786020016136035
\end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 2.803282 & 0.884316 & 0.576430 & 0.998670 \\ 0.884316 & 2.447266 & 0.600491 & 0.544755 \\ 0.576430 & 0.600491 & 2.995944 & 0.868780 \\ 0.998670 & 0.544755 & 0.868780 & 2.802353 \end{pmatrix} \begin{pmatrix} 0.263300 \\ -0.033386 \\ 0.093639 \\ -0.009786 \end{pmatrix} = \begin{pmatrix} 0.752782 \\ 0.202033 \\ 0.403762 \\ 0.298690 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
0.75278243304252\\
0.20203321079760816\\
0.4037621901806101\\
0.29869030143701125
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.7527824330425201\\
0.20203321079760822\\
0.40376219018061005\\
0.2986903014370113
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
-1.1102230246251565e-16\\
-5.551115123125783e-17\\
5.551115123125783e-17\\
-5.551115123125783e-17
\end{pmatrix} $$

In [100]:
def compare_result(name: str, val: Matrix, n: int = n) -> None:
    print_tex(name, '=', get_exact_column(val, n), r'\stackrel{?}{=}', get_exact_column(X_Gauss, n), '=X^*')
    print_tex(rf'||X^* - {name}||_\infty=', (X_Gauss - val).norm())

data = {}

def add_data(method_name: str, iterations: int, ans: Matrix) -> None:
    data[method_name] = {
        'Количество Итераций': str(iterations),
        'Фактическая Погрешность': str((X_Gauss - ans).norm())
    }

Преобразуем исходную систему к системе вида $x = H_Dx + g_D$
$$ H_D = E - D^{-1}A $$
$$ g_D = D^{-1}b $$


In [56]:
D, D_inv = Matrix(np.zeros((n, n))), Matrix(np.zeros((n, n)))

for i in range(n):
    D[i][i] = A[i][i]
    D_inv[i][i] = 1 / A[i][i]

H_D = Matrix(np.identity(n) - D_inv @ A)
g_D = Matrix(D_inv @ B)
print_tex('H_D=', H_D, ',~~', 'g_D=', g_D)
print_tex(rf'||H_D||_\infty = ', H_D.norm())


$$ H_D= \begin{pmatrix} 0 & -0.315457 & -0.205627 & -0.356250 \\ -0.361349 & 0 & -0.245372 & -0.222597 \\ -0.192403 & -0.200435 & 0 & -0.289985 \\ -0.356369 & -0.194392 & -0.310018 & 0 \end{pmatrix} ,~~ g_D= \begin{pmatrix} 0.268536 \\ 0.082555 \\ 0.134770 \\ 0.106586 \end{pmatrix} $$

$$ ||H_D||_\infty =  0.8773345284501621 $$

Найдем априорную оценку для того $k$, при котором $||x^* - x^k||_\infty < \varepsilon,~\varepsilon = 0.001$


In [57]:
def apr_eval(
    H: Matrix = H_D,
    g: Matrix = g_D,
    k: int = 1
) -> float:
    return H.norm()**k / (1 - H.norm()) * g.norm()


eps = 1.e-3
k = 1
while (apr_eval(k=k) >= eps):
    k += 1
k


59

In [101]:
def iterative_method(
    H: Matrix = H_D,
    G: Matrix = g_D,
    n: int = n
) -> Union[Matrix, Matrix, int]:
    next = Matrix(np.zeros((n, 1)))
    iter = 0
    while (X_Gauss - next).norm() >= eps:
        prev, next = next, H @ next + G
        iter += 1
    return next, prev, iter


iter_ans, prev, iter = iterative_method()
compare_result(rf'X^{({iter})}', iter_ans)


$$ X^{23} = \begin{pmatrix}
0.264127215250451\\
-0.03259676542638332\\
0.09431517287388672\\
-0.008979700214338096
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.263299596030353\\
-0.033386440745686745\\
0.09363947991217723\\
-0.009786020016136035
\end{pmatrix} =X^* $$

$$ ||X^* - X^{23}||_\infty= 0.000827619220097997 $$

In [102]:
add_data('Метод простой итерации', iter, iter_ans)

In [59]:
print(f'Априорная оценка при k={iter}:', apr_eval(k=iter))

Априорная оценка при k=23: 0.107914660126713


In [60]:
def apostr_eval(
    x_k: Matrix = iter_ans,
    x_prev: Matrix = prev,
    H: Matrix = H_D,
) -> float:
    return H.norm() / (1 - H.norm()) * (x_k - x_prev).norm()

print('Апостериорная оценка:', apostr_eval(iter_ans, prev))


Апостериорная оценка: 0.013173840840259856


In [61]:
def spec_radius(H: Matrix = H_D):
    return max(np.absolute(np.linalg.eigvals(H)))

rho = spec_radius(H_D)

def luster_approx(x_k: Matrix = iter_ans, x_prev: Matrix = prev, rho: float = rho) -> Matrix:
    return prev + (x_k - x_prev) / (1 - rho)

X_luster = luster_approx()

compare_result(rf'X^{({iter})}_\text{{lust}}', X_luster)


$$ X^{23}_\text{lust} = \begin{pmatrix}
0.27229330458830125\\
-0.024805077081811906\\
0.10098220899649399\\
-0.0010237827772007178
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.263299596030353\\
-0.033386440745686745\\
0.09363947991217723\\
-0.009786020016136035
\end{pmatrix} =X^* $$

$$ ||X^* - X^{23}_\text{lust}||_\infty= 0.008993708557948232 $$

Вычислим решение $x = H_Dx + g_D$ методом Зейделя с точностью $\varepsilon = 0.001$

In [103]:
def seid_method(
    H: Matrix = H_D,
    G: Matrix = g_D,
    n: int = n
) -> Union[Matrix, Matrix, int]:
    H_L = Matrix(np.array([[H[i][j] if i > j else 0 for j in range(n)] for i in range(n)]))
    H_R = Matrix(np.array([[H[i][j] if i <= j else 0 for j in range(n)] for i in range(n)]))
    H_seid = (np.identity(n) - H_L).inv() @ H_R
    g_seid = (np.identity(n) - H_L).inv() @ G
    next = Matrix(np.zeros((n, 1)))
    iter = 0
    while (X_Gauss - next).norm() >= eps:
        prev, next = next, H_seid @ next + g_seid
        iter += 1
    return next, prev, iter
    
seid_ans, seid_prev, seid_iter = seid_method()
compare_result(rf'X^{({seid_iter})}_\text{{seid}}', seid_ans)

$$ X^{4}_\text{seid} = \begin{pmatrix}
0.26303661864123007\\
-0.03342060384137316\\
0.0935808395876058\\
-0.009667482566584645
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.263299596030353\\
-0.033386440745686745\\
0.09363947991217723\\
-0.009786020016136035
\end{pmatrix} =X^* $$

$$ ||X^* - X^{4}_\text{seid}||_\infty= 0.00026297738912295365 $$

In [104]:
add_data('Метод Зейделя', seid_iter, seid_ans)

Получим решение системы $Ax = b$ методом верхней релаксации с точностью $\varepsilon = 0.001$

In [105]:
def rel_method(
    A: Matrix = A,
    b: Matrix = B,
    H: Matrix = H_D,
    G: Matrix = g_D,
    n: int = n
) -> Union[Matrix, Matrix, int]:
    rho = spec_radius(H)
    q_opt = 2 / (1 + np.sqrt(1 - rho**2))
    next = Matrix(np.zeros((n, 1)))
    iter = 0
    while (X_Gauss - next).norm() >= eps:
        prev = next
        for i in range(n):
            next[i][0] = prev[i][0] + q_opt * (b[i][0] - np.sum([next[j][0] * A[i][j] for j in range(
                i)]) - np.sum([prev[j][0] * A[i][j] for j in range(i, n)])) / A[i][i]
        iter += 1
    return next, prev, iter


rel_ans, rel_prev, rel_iter = rel_method()
compare_result(rf'X^{({rel_iter})}_\text{{rel}}', rel_ans)

$$ X^{5}_\text{rel} = \begin{pmatrix}
0.26407927387192404\\
-0.03273274057198576\\
0.09404027538629273\\
-0.010191239109528256
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.263299596030353\\
-0.033386440745686745\\
0.09363947991217723\\
-0.009786020016136035
\end{pmatrix} =X^* $$

$$ ||X^* - X^{5}_\text{rel}||_\infty= 0.0007796778415710137 $$

In [106]:
add_data('Метод верхней релаксации', rel_iter, rel_ans)

In [107]:
df = pd.DataFrame(data).transpose()
df

,Количество Итераций,Фактическая Погрешность
Метод простой итерации,23,0.000827619220097997
Метод Зейделя,4,0.00026297738912295365
Метод верхней релаксации,5,0.0007796778415710137
